In [ ]:
import numpy as np
np.set_printoptions(linewidth=100000000, formatter={'all': lambda x: str(x)})
# Test the type of distribution of data for any distribution
from scipy.stats import kstest
# Test if data is distributed in normal way
from scipy.stats import shapiro
# Test with parametric t-test
from scipy.stats import ttest_ind, ttest_rel
# Test with nonparametric test for comparing of two
from scipy.stats import wilcoxon 
# Test with nonparametric test for comparing of two or more
from scipy.stats import kruskal
# Test with nonparametric test for comparing of three or more
from scipy.stats import friedmanchisquare
# data preparation
from sklearn.preprocessing import normalize

[Nonparametric statistics](https://en.wikipedia.org/wiki/Nonparametric_statistics)

# Example of multiple runs on one problem

## Load data

In [ ]:
data = np.stack((np.loadtxt('jDE_1_30_v'), np.loadtxt('SCA_1_30_v'), np.loadtxt('BBFWA_1_30_v')))

## Normalize the data

In [ ]:
ndata = normalize(data)

## Test for normal distribution
[Kolmogorov–Smirnov](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test) test: Tests whether a sample is drawn from a given distribution, or whether two samples are drawn from the same distribution

In [ ]:
kstest?

In [ ]:
print(kstest(ndata[0], 'norm'))
print(kstest(ndata[1], 'norm'))
print(kstest(ndata[2], 'norm'))

[Shapiro–Wilk test](https://en.wikipedia.org/wiki/Shapiro%E2%80%93Wilk_test) is a test of normality in frequentist statistics.

In [ ]:
shapiro?

In [ ]:
print(shapiro(ndata[0]))
print(shapiro(ndata[1]))
print(shapiro(ndata[2]))

## Get ranks

In [ ]:
# TODO

## Run t-test related

In [ ]:
ttest_rel?

In [ ]:
print ('jDE vs. SCA:  \t', ttest_rel(ndata[0], ndata[1]))
print ('jDE vs. BBFWA:\t', ttest_rel(ndata[0], ndata[2]))

## Run Wilcoxon test
[Wilcoxon signed-rank](https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test) test: tests whether matched pair samples are drawn from populations with different mean ranks

In [ ]:
wilcoxon?

In [ ]:
print ('jDE vs. SCA:  \t', wilcoxon(ndata[0], ndata[1]))
print ('jDE vs. BBFWA:\t', wilcoxon(ndata[0], ndata[2]))

## Run Kruskal test
[Kruskal–Wallis one-way analysis of variance by ranks](https://en.wikipedia.org/wiki/Kruskal%E2%80%93Wallis_one-way_analysis_of_variance): tests whether > 2 independent samples are drawn from the same distribution

In [ ]:
kruskal?

In [ ]:
print ('jDE vs. SCA:  \t', kruskal(ndata[0], ndata[1]))
print ('jDE vs. BBFWA:\t', kruskal(ndata[0], ndata[2]))

## Run Friedman test
[Friedman two-way analysis of variance by ranks](https://en.wikipedia.org/wiki/Friedman_test): tests whether k treatments in randomized block designs have identical effects

In [ ]:
friedmanchisquare?

In [ ]:
print ('jDE vs. SCA vs. BBFWA: ', friedmanchisquare(ndata[0], ndata[1], ndata[2]))